口罩實名制在去年初開始實施，為了避免像之前那樣出現排隊搶購的情況，健保署規劃提供即時的口罩庫存資訊給民眾參考，此口罩地圖 API 網址合併了多種開放資料，包含著特約藥局、診所相關資訊，以及健保署提供的口罩剩餘數量明細。

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
# 收集
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
r = requests.get(url)
response = r.text
# 整理
data = json.loads(response)
with open("mask.json", "w") as f:
    f.write(json.dumps(data))
with open('mask.json', 'r') as f:
    data = json.load(f)

In [ ]:
#計算各地區的藥局數量
from collections import defaultdict
med_count = defaultdict(int)
for row in data ["features"]:
    #把縣市沒有登記到的醫院自動使用住址登記上
    if row['properties']['county'] == "":
        county = each["address"][0:3]
    else:
        conunty = row['properties']['county']
    med_count[conunty] += 1

print(med_count)

defaultdict(<class 'int'>, {'臺北市': 340, '高雄市': 424, '臺中市': 436, '臺南市': 280, '基隆市': 57, '新竹市': 39, '嘉義市': 67, '新北市': 507, '桃園市': 271, '新竹縣': 48, '宜蘭縣': 76, '苗栗縣': 57, '彰化縣': 181, '南投縣': 67, '雲林縣': 131, '嘉義縣': 84, '屏東縣': 140, '澎湖縣': 11, '花蓮縣': 47, '臺東縣': 23, '金門縣': 6, '連江縣': 1})


In [ ]:
# 計算每個地區的剩餘口罩數量(分成人的和小孩的)，並且從大到小排列
# 宣告 dictionary 用來存放資料
child_count = defaultdict(int)
adult_count = defaultdict(int) 
 
# 將資料迭代印出做計算
for row in data['features']:
    each = row["properties"]
    #把縣市沒有登記到的醫院自動使用住址登記上
    if each["county"] == "":
        county = each["address"][0:3]
    else:
        conunty = each['county']
    mask_child = each['mask_adult']
    mask_adult = each['mask_child']
    child_count[conunty] += mask_child
    adult_count[conunty] += mask_adult


# 排序
print(sorted(adult_count.items(), key=lambda kv: kv[1], reverse=True))
print(sorted(child_count.items(), key=lambda kv: kv[1], reverse=True))

[('新北市', 548660), ('臺中市', 489570), ('高雄市', 424980), ('臺北市', 344250), ('臺南市', 338410), ('桃園市', 276820), ('彰化縣', 222670), ('屏東縣', 169820), ('雲林縣', 168820), ('嘉義縣', 112370), ('嘉義市', 97820), ('宜蘭縣', 93670), ('南投縣', 91630), ('基隆市', 84380), ('苗栗縣', 83410), ('花蓮縣', 57840), ('新竹縣', 56040), ('新竹市', 43580), ('臺東縣', 25810), ('澎湖縣', 16510), ('金門縣', 14980), ('連江縣', 3470)]
[('新北市', 677660), ('臺中市', 556440), ('高雄市', 428940), ('臺南市', 408400), ('桃園市', 345600), ('臺北市', 323660), ('彰化縣', 219400), ('雲林縣', 199470), ('屏東縣', 184250), ('苗栗縣', 134180), ('嘉義縣', 113180), ('嘉義市', 107840), ('宜蘭縣', 106900), ('南投縣', 91240), ('基隆市', 80100), ('花蓮縣', 57330), ('新竹縣', 53760), ('新竹市', 45630), ('臺東縣', 28510), ('金門縣', 15510), ('澎湖縣', 12600), ('連江縣', 7930)]


In [ ]:
# 透過 Pandas 套件計算
properties = []

for info in data['features']:
    properties += [info['properties']]
    
#把縣市沒有登記到的醫院自動使用住址登記上
for each in properties:
    if each["county"] == "":
        each["county"] = each["address"][0:3]

properties = pd.DataFrame(properties)

total = properties[["county", "mask_adult", "mask_child"]]
total_sum = (total.groupby(by=['county']).sum()).sort_values(by=["mask_adult", "mask_child"], ascending=False)
print(total_sum)

        mask_adult  mask_child
county                        
新北市         677660      548660
臺中市         556440      489570
高雄市         428940      424980
臺南市         408400      338410
桃園市         345600      276820
臺北市         323660      344250
彰化縣         219400      222670
雲林縣         199470      168820
屏東縣         184250      169820
苗栗縣         134180       83410
嘉義縣         113180      112370
嘉義市         107840       97820
宜蘭縣         106900       93670
南投縣          91240       91630
基隆市          80100       84380
花蓮縣          57330       57840
新竹縣          53760       56040
新竹市          45630       43580
臺東縣          28510       25810
金門縣          15510       14980
澎湖縣          12600       16510
連江縣           7930        3470
